In [1]:
import pandas as pd
import datetime
import stock_data as stock
import numpy as np

scrapy = stock.Scrapy()

### 資料獲取

In [2]:
today = datetime.datetime.today()
today = datetime.datetime(2022, 12, 31)
start = f"{today.year}-01-01"
end = f"{today.year}-12-31"
# start = "2013-01-01"
# end = "2023-09-20"
start, end

('2022-01-01', '2022-12-31')

In [3]:
price = scrapy.get_price(
    start = start,
    end = end,
    mode = "listed"
)
price.to_csv("data/price.csv", index = False)
print(f"length = {len(price)}")
price.head()

------------------------------ Get ticker. ------------------------------
------------------------------ Get price. ------------------------------
[*********************100%%**********************]  976 of 976 completed


4 Failed downloads:
['6901.TW', '4569.TW', '6657.TW', '6863.TW']: Exception("%ticker%: Data doesn't exist for startDate = 1640966400, endDate = 1672416000")



------------------------------ Clean data. ------------------------------


  0%|          | 0/976 [00:00<?, ?it/s]c:\Users\tzuli\Documents\python\stock\stock_data.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Symbol"] = symbol#.replace(".TWO", "").replace(".TW", "")
c:\Users\tzuli\Documents\python\stock\stock_data.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Symbol"] = symbol#.replace(".TWO", "").replace(".TW", "")
 55%|█████▍    | 533/976 [00:01<00:01, 241.73it/s]c:\Users\tzuli\Documents\python\stock\stock_data.py:159: SettingWithCopyWarning: 
A value i

length = 238818


,Symbol,Date,Open,High,Low,Close,Adj Close,Volume
0,5484.TW,2022-01-03,16.00,16.10,15.80,15.95,15.95,11004.0
1,5484.TW,2022-01-04,15.80,16.20,15.80,16.20,16.20,27483.0
2,5484.TW,2022-01-05,16.00,16.15,16.00,16.15,16.15,21016.0
3,5484.TW,2022-01-06,16.00,16.05,16.00,16.05,16.05,19024.0
4,5484.TW,2022-01-07,15.85,16.15,15.85,16.15,16.15,8552.0


In [4]:
end = today.year - 1911 - 1
start = end - 2
# end = 111
# start = 100
start, end

(108, 110)

In [5]:
ratio = scrapy.get_financial_statement(
    type_ = 3,
    clean = 1, 
    mode = "listed",
    start_year = start, 
    end_year = end
)
ratio.to_csv("data/ratio.csv", index = False)
print(f"length = {len(ratio)}")
ratio.head()

------------------------------ Get Profit Analysis of sii. ------------------------------


100%|██████████| 3/3 [01:20<00:00, 26.75s/it]

length = 11061


,year,season,公司代號,公司名稱,營業收入 (百萬元),毛利率(%) (營業毛利)/ (營業收入),營業利益率(%) (營業利益)/ (營業收入),稅前純益率(%) (稅前純益)/ (營業收入),稅後純益率(%) (稅後純益)/ (營業收入)
0,108,1,1101,台泥,25356.33,25.59,19.85,22.1,16.68
1,108,1,1102,亞泥,19385.68,24.03,19.49,24.47,19.65
2,108,1,1103,嘉泥,452.38,14.42,-12.37,37.91,32.84
3,108,1,1104,環泥,1187.3,8.96,0.85,13.25,13.19
4,108,1,1108,幸福,834.35,3.2,-3.16,-4.2,-3.57


### 資料載入

In [6]:
price = pd.read_csv("data/price.csv")
ratio = pd.read_csv("data/ratio.csv")
print(f"price length = {len(price)}")
print(f"ratio length = {len(ratio)}")

price length = 238818
ratio length = 11061


### 資料清理

In [7]:
ratio = ratio.iloc[:, [0,1,2,3,4,5,6,8]]
ratio.columns = ['Year', 'Season', 'Symbol', 'Name', "Sales", "Gross", "Operating", "Net"]
ratio["Symbol"] = ratio["Symbol"].astype(str)
ratio.head()

,Year,Season,Symbol,Name,Sales,Gross,Operating,Net
0,108,1,1101,台泥,25356.33,25.59,19.85,16.68
1,108,1,1102,亞泥,19385.68,24.03,19.49,19.65
2,108,1,1103,嘉泥,452.38,14.42,-12.37,32.84
3,108,1,1104,環泥,1187.30,8.96,0.85,13.19
4,108,1,1108,幸福,834.35,3.20,-3.16,-3.57


In [8]:
price["Symbol"] = price["Symbol"].str.split(".").str[0]
price = price.drop(["Open", "High", "Low", "Close"], axis = 1)
price = price.rename(columns = {"Adj Close": "Close"})
price.head()

,Symbol,Date,Close,Volume
0,5484,2022-01-03,15.95,11004.0
1,5484,2022-01-04,16.20,27483.0
2,5484,2022-01-05,16.15,21016.0
3,5484,2022-01-06,16.05,19024.0
4,5484,2022-01-07,16.15,8552.0


### 基本面篩選

In [9]:
# 填補缺失年份和季度
ratio1 = pd.DataFrame()
g = ratio.groupby("Symbol")
groups = ratio["Symbol"].unique()
for group in groups:
    df_group = g.get_group(group)
    combinations = ratio[["Year", "Season"]].value_counts().index
    combinations = combinations.sort_values().to_frame(index = False).iloc[:-1] # Q4要隔年3月才會公布
    df_group = pd.merge(combinations, df_group, on = ["Year", "Season"], how = "left")
    ratio1 = pd.concat([ratio1, df_group], ignore_index = True)
ratio1[["Symbol", "Name"]] = ratio1[["Symbol", "Name"]].fillna(method = "ffill")
ratio = ratio1.copy()

print(f"ratio length = {len(ratio)}")
ratio.head()

ratio length = 10483


,Year,Season,Symbol,Name,Sales,Gross,Operating,Net
0,108,1,1101,台泥,25356.33,25.59,19.85,16.68
1,108,2,1101,台泥,56723.49,27.87,22.53,20.91
2,108,3,1101,台泥,87561.88,29.16,23.86,21.59
3,108,4,1101,台泥,122783.01,29.25,24.56,20.73
4,109,1,1101,台泥,21976.20,26.30,20.30,15.84


In [10]:
# 營收YOY為正，且三率三升
ratio1 = pd.DataFrame()
g = ratio.groupby("Symbol")
groups = ratio["Symbol"].unique()
for group in groups:
    df_group = g.get_group(group)
    
    df_group = df_group.iloc[-8:].reset_index(drop = True)
    remove_flag = (df_group.loc[:, "Sales":"Net"] < 0).any().any()
    if remove_flag:
        continue
    
    df_group["Sales_pct"] = df_group["Sales"].pct_change(4).values    
    remove_flag = (df_group["Sales_pct"] < 0).any()
    if remove_flag:
        continue
    
    keep_flag = (df_group.loc[:3, "Sales":"Net"] < df_group.shift(-4).loc[:3, "Sales":"Net"]).all().all()
    if keep_flag:
        df_group = df_group.iloc[-4:].reset_index(drop = True)
        ratio1 = pd.concat([ratio1, df_group], ignore_index = True)
ratio = ratio1.copy()

print(f"ratio length = {len(ratio)}")
ratio.head()

ratio length = 208


,Year,Season,Symbol,Name,Sales,Gross,Operating,Net,Sales_pct
0,109,4,1513,中興電,15390.79,21.35,11.18,10.08,0.243447
1,110,1,1513,中興電,4682.71,26.83,17.04,13.16,0.538761
2,110,2,1513,中興電,8744.14,24.83,14.69,10.71,0.369947
3,110,3,1513,中興電,12863.56,23.50,14.39,10.69,0.295812
4,109,4,1720,生達,4305.40,44.59,16.41,14.53,0.093538


In [11]:
keep_symbol = ratio["Symbol"].unique()
print(f"length keep_symbol = {len(keep_symbol)}")
print(f"keep_symbol = {keep_symbol}")

price = price.query("Symbol in @keep_symbol")
price = price.sort_values(["Symbol", "Date"]).reset_index(drop = True)
print(f"price length = {len(price)}")

length keep_symbol = 52
keep_symbol = ['1513' '1720' '2010' '2032' '2108' '2303' '2353' '2357' '2368' '2376'
 '2377' '2382' '2393' '2441' '2454' '2472' '2481' '2484' '2597' '2615'
 '3006' '3010' '3017' '3034' '3037' '3042' '3056' '3257' '3305' '3312'
 '3515' '3530' '3711' '4555' '4576' '4952' '4961' '5469' '5538' '5871'
 '6168' '6183' '6191' '6271' '6582' '6591' '6669' '8016' '8046' '8081'
 '8261' '9921']
price length = 12792


### 技術面篩選

In [12]:
total_return = []
price1 = pd.DataFrame()
g = price.groupby("Symbol")
groups = price["Symbol"].unique()
for group in groups:
    df_group = g.get_group(group)
    start = df_group["Close"].iloc[0]
    end = df_group["Close"].iloc[-1]
    return_ = round((end - start) / start * 100, 2)
    # if start < 100:
    total_return.append(return_)
    print(f"Symbol: {group} ({return_}%)\n年初: {start}, 年底: {end}\n")
    # break

Symbol: 1513 (57.89%)
年初: 41.51, 年底: 65.54

Symbol: 1720 (51.07%)
年初: 39.1, 年底: 59.07

Symbol: 2010 (-30.29%)
年初: 22.12, 年底: 15.42

Symbol: 2032 (-8.52%)
年初: 21.94, 年底: 20.07

Symbol: 2108 (-51.76%)
年初: 73.14, 年底: 35.28

Symbol: 2303 (-31.65%)
年初: 55.52, 年底: 37.95

Symbol: 2353 (-3.55%)
年初: 22.23, 年底: 21.44

Symbol: 2357 (-17.21%)
年初: 308.76, 年底: 255.61

Symbol: 2368 (3.56%)
年初: 80.96, 年底: 83.84

Symbol: 2376 (-21.78%)
年初: 133.31, 年底: 104.28

Symbol: 2377 (-18.61%)
年初: 141.17, 年底: 114.9

Symbol: 2382 (-16.32%)
年初: 80.59, 年底: 67.44

Symbol: 2393 (-24.33%)
年初: 46.45, 年底: 35.15

Symbol: 2441 (-32.67%)
年初: 66.86, 年底: 45.02

Symbol: 2454 (-40.25%)
年初: 944.7, 年底: 564.49

Symbol: 2472 (-15.18%)
年初: 59.41, 年底: 50.39

Symbol: 2481 (-42.06%)
年初: 96.08, 年底: 55.67

Symbol: 2484 (-10.3%)
年初: 35.25, 年底: 31.62

Symbol: 2597 (13.3%)
年初: 101.46, 年底: 114.95

Symbol: 2615 (-45.88%)
年初: 135.47, 年底: 73.31

Symbol: 3006 (-56.58%)
年初: 146.37, 年底: 63.56

Symbol: 3010 (-15.61%)
年初: 92.38, 年底: 77.96

Symbol: 30

In [13]:
round(np.mean(total_return), 2)

-22.32

In [14]:
{
    "109": {
        "counts": 19,
        "return": 16.66,
    },
    "110": {
        "counts": 17,
        "return": 21.04
    },
    "111": {
        "counts": 48,
        "return": -13.69
    },
    "112": {
        "counts": 49,
        "return": 27.01
    },
}

{'109': {'counts': 19, 'return': 16.66},
 '110': {'counts': 17, 'return': 21.04},
 '111': {'counts': 48, 'return': -13.69},
 '112': {'counts': 49, 'return': 27.01}}